In [ ]:
import json
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection

# -----------------------------
# Cargar GeoJSON de CCAA España
# -----------------------------
with open("ccaa_espana.geojson", "r", encoding="utf-8") as f:
    geo = json.load(f)

# -----------------------------
# Datos (reconstruidos del mapa)
# -----------------------------
values = {
    "Galicia": 72.0,
    "PrincipadodeAsturias": 72.0,
    "Cantabria": 73.0,
    "PaísVasco": 65.0,
    "ComunidadForaldeNavarra": 72.0,
    "LaRioja": 76.0,
    "Aragón": 72.0,
    "Cataluña": 65.0,
    "CastillayLeón": 74.8,
    "ComunidaddeMadrid": 73.0,
    "Castilla-LaMancha": 76.0,
    "Extremadura": 76.0,
    "ComunidadValenciana": 73.0,
    "RegióndeMurcia": 74.8,
    "Andalucía": 72.0,
    "IslasBaleares": 65.0,
    "Canarias": 65.0
}

# Clases exactas del original
bins = [61.6, 67.3, 72.3, 73.8, 75.4, 77.1]

# Escala de grises manual
colors = ["#f0f0f0", "#cccccc", "#999999", "#666666", "#333333"]

def classify(v):
    for i in range(len(bins) - 1):
        if bins[i] <= v < bins[i + 1]:
            return colors[i]
    return colors[-1]

# -----------------------------
# Configuración editorial
# -----------------------------
plt.rcParams.update({
    "font.family": "serif",
    "font.size": 10
})

fig, ax = plt.subplots(figsize=(7.5, 6))

patches = []
facecolors = []

# -----------------------------
# Dibujar polígonos
# -----------------------------
for feature in geo["features"]:
    name = feature["properties"]["NAME_1"]
    val = values.get(name)

    if val is None:
        continue

    geom = feature["geometry"]

    if geom["type"] == "Polygon":
        coords = geom["coordinates"][0]
        patches.append(Polygon(coords, closed=True))
        facecolors.append(classify(val))

    elif geom["type"] == "MultiPolygon":
        for poly in geom["coordinates"]:
            coords = poly[0]
            patches.append(Polygon(coords, closed=True))
            facecolors.append(classify(val))

collection = PatchCollection(
    patches,
    facecolor=facecolors,
    edgecolor="black",
    linewidth=0.8
)

ax.add_collection(collection)

# Ajustes de vista
ax.autoscale()
ax.set_axis_off()

# -----------------------------
# Leyenda (estilo académico)
# -----------------------------
legend_labels = [
    "75,4 a 77,1",
    "73,8 a 75,4",
    "72,3 a 73,8",
    "67,3 a 72,3",
    "61,6 a 67,3"
]

legend_patches = [
    plt.Line2D([0], [0], marker='s', color='black',
               markerfacecolor=colors[::-1][i],
               markersize=10, linestyle='')
    for i in range(5)
]

ax.legend(
    legend_patches,
    legend_labels,
    title="España\nElecciones 2000",
    loc="lower right",
    frameon=True,
    fancybox=False,
    edgecolor="black"
)

plt.tight_layout()

# Exportar PNG
plt.savefig(
    "mapa_espana_ccaa_part.png",
    dpi=300,
    bbox_inches="tight"
)

plt.show()
